In [1]:
import sys
sys.path.append('../')

import numpy as np
import networkx as nx
import itertools
import torch
import torch.nn.functional as F
import torch_sparse

from torch import nn
from models.convs import SheafConv, Z2SheafConv
from lib.laplace import build_norm_sheaf_laplacian, remove_duplicate_edges, build_sheaf_difussion_matrix, dirichlet_energy
from torch_geometric.nn.dense.linear import Linear
from scipy import linalg
from torch_geometric.utils import to_dense_adj, from_networkx
from torch_geometric.nn.conv import GCNConv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_dtype(torch.float64)

In [2]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)

In [3]:
class Z2SheafNet(torch.nn.Module):
    def __init__(self, layers=2):
        super().__init__()
        self.layers = layers
        self.linear = nn.Linear(dataset.num_node_features, 16)
        
        self.convs = nn.ModuleList()
        for i in range(layers):
            self.convs.append(Z2SheafConv(16, 16))
        self.final = nn.Linear(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.dropout(x, training=self.training)
        x = F.relu(self.linear(x))
        x = F.dropout(x, training=self.training)
        raw = x 
        
        for i in range(self.layers):
            x = self.convs[i](x, edge_index)
            x = 0.1 * raw + x

        x = self.final(x)
        return F.log_softmax(x, dim=1)
    
    
class GCN(torch.nn.Module):
    def __init__(self, layers=2):
        super().__init__()
        self.layers = layers
        self.linear = nn.Linear(dataset.num_node_features, 16)
        
        self.convs = nn.ModuleList()
        for i in range(layers):
            self.convs.append(GCNConv(16, 16))
        self.final = nn.Linear(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        x = F.relu(self.linear(x))
        x = F.dropout(x, training=self.training)
        raw = x 
        
        for i in range(self.layers):
            x = F.relu(self.convs[i](x, edge_index))
            x = raw + x
        
        x = self.final(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Z2SheafNet(layers=4)
# model = GCN(layers=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss.item()}")

Epoch: 0 | Loss: 1.9540689430955638
Epoch: 10 | Loss: 1.833361298368214
Epoch: 20 | Loss: 1.3535748969556007
Epoch: 30 | Loss: 1.0140017449221617
Epoch: 40 | Loss: 0.7277496079755661
Epoch: 50 | Loss: 0.5964785909918624
Epoch: 60 | Loss: 0.4002771864796739
Epoch: 70 | Loss: 0.36511657592418806
Epoch: 80 | Loss: 0.27248248233122785
Epoch: 90 | Loss: 0.32955938588210065
Epoch: 100 | Loss: 0.26374631539830484
Epoch: 110 | Loss: 0.15276595453794034
Epoch: 120 | Loss: 0.11843103081538953
Epoch: 130 | Loss: 0.0837491472851068
Epoch: 140 | Loss: 0.11864420132574112
Epoch: 150 | Loss: 0.1044953118913567
Epoch: 160 | Loss: 0.08820385205657091
Epoch: 170 | Loss: 0.04280260304945667
Epoch: 180 | Loss: 0.0950202715401205
Epoch: 190 | Loss: 0.08070941190772443


In [7]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print('Test Accuracy: {:.4f}'.format(acc))
correct = (pred[data.train_mask] == data.y[data.train_mask]).sum()
acc = int(correct) / int(data.train_mask.sum())
print('Train Accuracy: {:.4f}'.format(acc))

Test Accuracy: 0.7370
Train Accuracy: 1.0000
